# Hackaton

In [1]:
import folium
import json
import pandas as pd 
import geopandas as gpd
from shapely.geometry import Point
from gpt import parse_data, get_insights_of_disasters, get_insights_of_disasters_and_hospitals
from helpers.get_geojson import get_geojson
from helpers.polygons import is_inside

In [30]:
from IPython.display import display, HTML

def nice_print(string):
  html_string = ""
  string_array = string.split(' ')
  for i in range(0, len(string_array), 16):
    string_line = " ".join(string_array[i: i + 16])
    html_string += f'<p style="font-size: 16px";>{string_line}</p>\n'
  display(HTML(html_string))

## Load data

In [31]:
disasters_df = gpd.read_file('data/distasters_parsed.geojson')
hospitals_df = pd.read_csv('data/centros_salud_filtered_urgencia.csv')

In [128]:
disasters_df['Disaster Type'] = disasters_df['Disaster Type'].replace({"Flood": "Inundacion",
                                          "Volcanic activity": "Volcan",
                                          "Storm": "Tormenta",
                                          "Earthquake": "Terremoto",
                                          "Landslide": "Aluvion",
                                          "Wildfire": "Incendio"})

## Main Loop

- read input
- parse to structured
- get geojson
- filter type
- show data

In [129]:
disasters_df.head()

,Year,Seq,Glide,Disaster Group,Disaster Subgroup,Disaster Type,Disaster Subtype,Disaster Subsubtype,Event Name,Country,...,No Homeless,Total Affected,Insured Damages ('000 US$),Total Damages ('000 US$),CPI,Adm Level,Admin1 Code,Admin2 Code,Geo Locations,geometry
0,1964,17,NaN,Natural,Geophysical,Volcan,Ash fall,NaN,NaN,Chile,...,NaN,2000.0,NaN,NaN,11.984281,NaN,NaN,NaN,NaN,POINT (-72.23078 -39.28201)
1,1965,40,NaN,Natural,Hydrological,Inundacion,NaN,NaN,NaN,Chile,...,25000.0,375000.0,NaN,10000.0,12.174253,NaN,NaN,NaN,NaN,POINT (-70.39750 -23.65093)
2,1968,9008,NaN,Natural,Climatological,Drought,Drought,NaN,NaN,Chile,...,NaN,120000.0,NaN,55000.0,13.439657,NaN,NaN,NaN,NaN,POINT (-71.54297 -35.67515)
3,1968,57,NaN,Natural,Meteorological,Tormenta,NaN,NaN,NaN,Chile,...,NaN,NaN,NaN,NaN,13.439657,NaN,NaN,NaN,NaN,POINT (-71.54297 -35.67515)
4,1974,27,NaN,Natural,Hydrological,Inundacion,NaN,NaN,NaN,Chile,...,NaN,40000.0,NaN,10000.0,19.051854,NaN,NaN,NaN,NaN,POINT (-71.54297 -35.67515)


In [130]:
hospitals_df.head()

,X,Y,FID,codest,nomeste,codreg,nomreg,codss,nomss,codcom,...,Place,certifi,depend,nivel,viaacceso,direcc,Longitude,Latitude,cod_nuevo,Coordinates
0,-70.290543,-18.437603,15,01-900,SAPU-Dr. Amador Neghme de Arica,15,De Arica Parinacota,1,Servicio de Salud Arica,15101,...,Servicio de Atencion Primaria de Urgencia,S/C,Municipal,Primario,Avenida,Cancha Rayada 4149 Arica,-70.290543,-18.437603,200094,POINT (-70.2905433131 -18.4376026272)
1,-70.282444,-18.483881,16,01-901,SAPU-Iris Véliz Hume,15,De Arica Parinacota,1,Servicio de Salud Arica,15101,...,Servicio de Atencion Primaria de Urgencia,S/C,Municipal,Primario,Calle,Barros Luco 2269 Arica,-70.282444,-18.483881,200095,POINT (-70.2824436462 -18.4838808045)
2,-70.134674,-20.226259,34,02-800,SAPU-Cirujano Aguirre,1,De Tarapacá,2,Servicio de Salud Iquique,1101,...,Servicio de Atencion Primaria de Urgencia,S/C,Municipal,Primario,Pasaje,Chintaguay S/N Iquique,-70.134674,-20.226259,102800,POINT (-70.1346740237 -20.2262589719)
3,-70.150170,-20.208263,35,02-801,SAPU-Cirujano Videla,1,De Tarapacá,2,Servicio de Salud Iquique,1101,...,Servicio de Atencion Primaria de Urgencia,S/C,Municipal,Primario,Calle,Obispo Labbé S/N Iquique,-70.150170,-20.208263,102801,POINT (-70.150170342 -20.2082628936)
4,-70.136451,-20.236900,36,02-802,SAPU-Cirujano Guzmán,1,De Tarapacá,2,Servicio de Salud Iquique,1101,...,Servicio de Atencion Primaria de Urgencia,S/C,Municipal,Primario,Avenida,Pedro Prado 2736 Iquique,-70.136451,-20.236900,102802,POINT (-70.1364509765 -20.2369000291)


In [131]:
def plot_map(disasters_filtered, hospitals_filtered):
  m = folium.Map(location=[disasters_df['Latitude'].mean(), disasters_df['Longitude'].mean()], zoom_start=6)
  latitudes, longitudes = [], []
  for i in range(0, len(disasters_filtered)):
    latitudes.append(disasters_filtered.iloc[i]['Latitude'])
    longitudes.append(disasters_filtered.iloc[i]['Longitude'])
    folium.Marker([disasters_filtered.iloc[i]['Latitude'], disasters_filtered.iloc[i]['Longitude']], icon=folium.Icon(color='red'), popup=disasters_filtered.iloc[i]['Disaster Type']).add_to(m)
  for i in range(0, len(hospitals_filtered)):
    latitudes.append(hospitals_filtered.iloc[i]['Latitude'])
    longitudes.append(hospitals_filtered.iloc[i]['Longitude'])
    folium.Marker([hospitals_filtered.iloc[i]['Latitude'], hospitals_filtered.iloc[i]['Longitude']], icon=folium.Icon(color='blue'), popup=hospitals_filtered.iloc[i]['nomeste']).add_to(m)
  min_lat = min(latitudes)
  max_lat = max(latitudes)
  min_long = min(longitudes)
  max_long = max(longitudes)
  bounds = [[min_lat, min_long], [max_lat, max_long]]
  m.fit_bounds(bounds)
  return m

In [132]:
def location_insights(ubicacion):
    geojson = get_geojson(ubicacion)
    boolean_filter = []
    for _, row in disasters_df.iterrows():
        is_in = False
        for _, zone in geojson.iterrows():
            if zone['geometry'].contains(row['geometry']):
                boolean_filter.append(True)
                is_in = True
                break
        if not is_in:
            boolean_filter.append(False)

    filtered_disasters = disasters_df[boolean_filter]
    if len(filtered_disasters) == 0:
        print('No hay datos en la zona:', geojson)
        return
    disasters = filtered_disasters['Disaster Type'].value_counts()
    disasters_str = ""
    for index, value in disasters.items():
        disasters_str += f"{index}: {value}\n"
    print(disasters_str)
    nice_print(get_insights_of_disasters(disasters_str, ubicacion))
    return plot_map(filtered_disasters, [])

In [133]:
def calculate_distance(point1, point2):
    point1 = point1.strip('POINT ').strip('(').strip(')')
    x, y = point1.split()
    x, y = float(x), float(y)
    point1 = Point(x, y)
    return point1.distance(point2)

def get_nearest_hospitals(polygon):
    center = polygon.centroid
    point = Point(center.x, center.y)
    hosp_copy = hospitals_df.copy()
    hosp_copy['Distance'] = hosp_copy['Coordinates'].apply(lambda x: calculate_distance(x, point))
    nearest_row = hosp_copy.loc[hosp_copy['Distance'].idxmin()]
    return nearest_row

In [134]:
from shapely.geometry import Point, Polygon

def get_location_and_assist(ubicacion):
    geojson = get_geojson(ubicacion)
    boolean_filter = []
    boolean_filter_2 = []
    for _, row in disasters_df.iterrows():
        is_in = False
        for _, zone in geojson.iterrows():
            if zone['geometry'].contains(row['geometry']):
                boolean_filter.append(True)
                is_in = True
                break
        if not is_in:
            boolean_filter.append(False)
    for _, row in hospitals_df.iterrows():
        is_in = False
        for _, zone in geojson.iterrows():
            point = Point(row['Longitude'], row['Latitude'])
            if zone['geometry'].contains(point):
                boolean_filter_2.append(True)
                is_in = True
                break
        if not is_in:
            boolean_filter_2.append(False)
    filtered_disasters = disasters_df[boolean_filter]
    filtered_hospitals = hospitals_df[boolean_filter_2]
    print('hospitales:', len(filtered_hospitals))
    if len(filtered_disasters) == 0:
        print('No hay datos en la zona:', geojson)
        return
    disasters = filtered_disasters['Disaster Type'].value_counts()
    disasters_str = ""
    total_disasters = 0
    for index, value in disasters.items():
        disasters_str += f"{index}: {value}\n"
        total_disasters += value
    if len(filtered_hospitals) == 0:
        ratio = 0
    else:
        ratio = len(filtered_hospitals) / total_disasters
    print('Ratio desastres/centros:', ratio)
    print(disasters_str)
    if len(filtered_hospitals) == 0:
        hosp = get_nearest_hospitals(geojson)
        filtered_hospitals = pd.DataFrame([hosp])
    nice_print(get_insights_of_disasters_and_hospitals(disasters_str, ubicacion, ratio))
    return plot_map(filtered_disasters, filtered_hospitals)

In [155]:
queries = [
  'estoy en chaiten, hablame sobre los desastres naturales y los hospitales',
  'por favor hablame sobre que peligros hay en antofagasta, y que centros asistenciales me pueden ayudar',
  'cuanto miden los gatos en promedio',
  'estoy yendo a puerto mont. de que me debería preocupar',
  'voy a punta arenas hospitales',
]

In [156]:
_input = queries[4]
# _input = input('Ingrese query:')
parsed_input = parse_data(_input)
print(parsed_input)
ubicacion = parsed_input['ubicacion']

# location filter

if ubicacion and parsed_input['centro_asistencial']:
    m = get_location_and_assist(ubicacion)
elif ubicacion:
    m = location_insights(ubicacion)
else:
    print('Esa no es una query valida.')
    m = None

m

{'ubicacion': 'punta arenas', 'tipo_de_desastre': None, 'centro_asistencial': 'hospital'}
hospitales: 3
Ratio desastres/centros: 0.75
Inundacion: 2
Incendio: 2



In [ ]:
# Load GeoJSON data
with open("data/establecimientos_salud.geojson") as f:
    geo_json_data = json.load(f)

# Add the GeoJSON layer
folium.GeoJson(geo_json_data, name='geojson').add_to(m)